<a href="https://colab.research.google.com/github/Niv0902/Shablool/blob/main/LAb7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "QumyS_VYsudP",
        "outputId": "5a0825ad-2d80-44f3-e98a-bdc302a719d0"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Added documents: 1, 2\n",
            "Query results: {'id': '1', 'terms': ['cloud', 'computing'], 'results': ['1', '2'], 'timestamp': 'now'}\n",
            "Formatted results: {'id': '1', 'query_id': '1', 'formatted_results': [{'doc_id': '1', 'title': 'Python Programming', 'snippet': 'Python is a popular programming language for cloud computing...'}, {'doc_id': '2', 'title': 'Cloud Services', 'snippet': 'Cloud computing enables scalable microservices architecture...'}], 'count': 2}\n"
          ]
        }
      ],
      "source": [
        "# user_service.py\n",
        "class UserService:\n",
        "    def __init__(self):\n",
        "        self.users = {\n",
        "            '1': {'id': '1', 'name': 'John Doe', 'email': 'john@example.com'},\n",
        "            '2': {'id': '2', 'name': 'Jane Doe', 'email': 'jane@example.com'}\n",
        "        }\n",
        "\n",
        "    def get_user(self, user_id):\n",
        "        return self.users.get(user_id, {})\n",
        "\n",
        "# index_service.py\n",
        "class IndexService:\n",
        "    def __init__(self):\n",
        "        self.documents = {}\n",
        "        self.index = {}\n",
        "\n",
        "    def add_document(self, doc_data):\n",
        "        \"\"\"Add a document to the index\"\"\"\n",
        "        doc_id = str(len(self.documents) + 1)\n",
        "        self.documents[doc_id] = {**doc_data, 'id': doc_id}\n",
        "\n",
        "        # Create inverted index\n",
        "        words = doc_data['content'].lower().split()\n",
        "        for word in words:\n",
        "            if word not in self.index:\n",
        "                self.index[word] = set()\n",
        "            self.index[word].add(doc_id)\n",
        "\n",
        "        return self.documents[doc_id]\n",
        "\n",
        "    def get_document(self, doc_id):\n",
        "        \"\"\"Retrieve a document by ID\"\"\"\n",
        "        return self.documents.get(doc_id)\n",
        "\n",
        "    def search_word(self, word):\n",
        "        \"\"\"Find documents containing a word\"\"\"\n",
        "        word = word.lower()\n",
        "        return list(self.index.get(word, set()))\n",
        "\n",
        "# query_service.py\n",
        "class QueryService:\n",
        "    def __init__(self, index_service):\n",
        "        self.index_service = index_service\n",
        "        self.queries = {}\n",
        "\n",
        "    def create_query(self, query_data):\n",
        "        \"\"\"Create and execute a search query\"\"\"\n",
        "        try:\n",
        "            query_id = str(len(self.queries) + 1)\n",
        "            search_terms = query_data['terms']\n",
        "\n",
        "            # Find matching documents\n",
        "            results = set()\n",
        "            for term in search_terms:\n",
        "                doc_ids = self.index_service.search_word(term)\n",
        "                if not results:\n",
        "                    results = set(doc_ids)\n",
        "                else:\n",
        "                    results &= set(doc_ids)\n",
        "\n",
        "            # Create query record\n",
        "            query = {\n",
        "                'id': query_id,\n",
        "                'terms': search_terms,\n",
        "                'results': list(results),\n",
        "                'timestamp': query_data.get('timestamp', 'now')\n",
        "            }\n",
        "            self.queries[query_id] = query\n",
        "            return query\n",
        "\n",
        "        except Exception as e:\n",
        "            return {'error': str(e)}\n",
        "\n",
        "# result_service.py\n",
        "class ResultService:\n",
        "    def __init__(self, index_service, query_service):\n",
        "        self.index_service = index_service\n",
        "        self.query_service = query_service\n",
        "        self.results = {}\n",
        "\n",
        "    def format_results(self, query_id):\n",
        "        \"\"\"Format search results for display\"\"\"\n",
        "        try:\n",
        "            query = self.query_service.queries.get(query_id)\n",
        "            if not query:\n",
        "                return {'error': 'Query not found'}\n",
        "\n",
        "            formatted_results = []\n",
        "            for doc_id in query['results']:\n",
        "                doc = self.index_service.get_document(doc_id)\n",
        "                if doc:\n",
        "                    formatted_results.append({\n",
        "                        'doc_id': doc_id,\n",
        "                        'title': doc['title'],\n",
        "                        'snippet': doc['content'][:100] + '...'\n",
        "                    })\n",
        "\n",
        "            result_id = str(len(self.results) + 1)\n",
        "            result = {\n",
        "                'id': result_id,\n",
        "                'query_id': query_id,\n",
        "                'formatted_results': formatted_results,\n",
        "                'count': len(formatted_results)\n",
        "            }\n",
        "            self.results[result_id] = result\n",
        "            return result\n",
        "\n",
        "        except Exception as e:\n",
        "            return {'error': str(e)}\n",
        "\n",
        "# main.py\n",
        "def main():\n",
        "    # Initialize services\n",
        "    index_service = IndexService()\n",
        "    query_service = QueryService(index_service)\n",
        "    result_service = ResultService(index_service, query_service)\n",
        "\n",
        "    # Add sample documents\n",
        "    doc1 = index_service.add_document({\n",
        "        'title': 'Python Programming',\n",
        "        'content': 'Python is a popular programming language for cloud computing'\n",
        "    })\n",
        "    doc2 = index_service.add_document({\n",
        "        'title': 'Cloud Services',\n",
        "        'content': 'Cloud computing enables scalable microservices architecture'\n",
        "    })\n",
        "    print(f\"Added documents: {doc1['id']}, {doc2['id']}\")\n",
        "\n",
        "    # Create and execute a search query\n",
        "    query = query_service.create_query({\n",
        "        'terms': ['cloud', 'computing']\n",
        "    })\n",
        "    print(f\"Query results: {query}\")\n",
        "\n",
        "    # Format the results\n",
        "    formatted_results = result_service.format_results(query['id'])\n",
        "    print(f\"Formatted results: {formatted_results}\")\n",
        "\n",
        "if __name__ == \"__main__\":\n",
        "    main()\n",
        "\n",
        "# Example test cases\n",
        "def test_search():\n",
        "    # Initialize services\n",
        "    index_service = IndexService()\n",
        "    query_service = QueryService(index_service)\n",
        "    result_service = ResultService(index_service, query_service)\n",
        "\n",
        "    # Test document indexing\n",
        "    doc = index_service.add_document({\n",
        "        'title': 'Test Document',\n",
        "        'content': 'This is a test document about microservices'\n",
        "    })\n",
        "    assert doc['id'] == '1'\n",
        "\n",
        "    # Test search functionality\n",
        "    query = query_service.create_query({\n",
        "        'terms': ['test', 'microservices']\n",
        "    })\n",
        "    assert len(query['results']) == 1\n",
        "\n",
        "    # Test result formatting\n",
        "    results = result_service.format_results(query['id'])\n",
        "    assert results['count'] == 1\n",
        "    assert 'test document' in results['formatted_results'][0]['snippet'].lower()\n",
        "\n",
        "if __name__ == \"__main__\":\n",
        "    test_search()"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "\n",
        "\n",
        "\"\"\"\n",
        "Function as a Service (FaaS) Architecture Demo: Search Engine Implementation\n",
        "\n",
        "This code demonstrates key FaaS principles through a simple search engine implementation.\n",
        "It simulates how serverless functions would operate in a cloud environment.\n",
        "\n",
        "Key FaaS Characteristics Demonstrated:\n",
        "1. Stateless Functions - Each invocation is independent\n",
        "2. Event-Driven Architecture - Functions respond to specific triggers\n",
        "3. Single Responsibility - Each function performs one specific task\n",
        "4. Automatic Scaling (simulated) - Functions can handle multiple concurrent requests\n",
        "\"\"\"\n",
        "\n",
        "class IndexerFunction:\n",
        "\n",
        "    def __init__(self):\n",
        "        # In real FaaS, this would be external storage\n",
        "        self.index = {}\n",
        "\n",
        "    def handle(self, event):\n",
        "        \"\"\"\n",
        "        Function entry point - similar to AWS Lambda handler\n",
        "\n",
        "        Args:\n",
        "            event (dict): Contains document to be indexed\n",
        "                {\n",
        "                    'document_id': str - Unique document identifier\n",
        "                    'content': str - Document content to index\n",
        "                }\n",
        "\n",
        "        Returns:\n",
        "            dict: Indexing operation results\n",
        "                {\n",
        "                    'status': str - Operation status\n",
        "                    'indexed_words': int - Number of words processed\n",
        "                }\n",
        "        \"\"\"\n",
        "        doc_id = event['document_id']\n",
        "        content = event['content'].lower().split()\n",
        "\n",
        "        # Build inverted index\n",
        "        for word in content:\n",
        "            if word not in self.index:\n",
        "                self.index[word] = set()\n",
        "            self.index[word].add(doc_id)\n",
        "\n",
        "        return {\n",
        "            'status': 'success',\n",
        "            'indexed_words': len(content)\n",
        "        }\n",
        "\n",
        "class SearcherFunction:\n",
        "    \"\"\"\n",
        "    Simulates a FaaS search function.\n",
        "\n",
        "    FaaS Characteristics:\n",
        "    - Event-driven: Responds to search requests\n",
        "    - Stateless: Each search is independent\n",
        "    - Scalable: Multiple instances can handle concurrent searches\n",
        "\n",
        "\n",
        "    \"\"\"\n",
        "    def __init__(self, index_service):\n",
        "        self.index = index_service.index\n",
        "\n",
        "    def handle(self, event):\n",
        "        \"\"\"\n",
        "        Search function entry point\n",
        "\n",
        "        Args:\n",
        "            event (dict): Contains search parameters\n",
        "                {\n",
        "                    'query': str - Search terms\n",
        "                }\n",
        "\n",
        "        Returns:\n",
        "            dict: Search results\n",
        "                {\n",
        "                    'status': str - Operation status\n",
        "                    'results': list - Matching document IDs\n",
        "                }\n",
        "\n",
        "        Note: In real FaaS:\n",
        "        - Would include error handling\n",
        "        - Would implement timeouts\n",
        "        - Would include logging/monitoring\n",
        "        \"\"\"\n",
        "        terms = event['query'].lower().split()\n",
        "        results = set()\n",
        "\n",
        "        for term in terms:\n",
        "            if term in self.index:\n",
        "                if not results:\n",
        "                    results = self.index[term].copy()\n",
        "                else:\n",
        "                    results &= self.index[term]\n",
        "\n",
        "        return {\n",
        "            'status': 'success',\n",
        "            'results': list(results)\n",
        "        }\n",
        "\n",
        "class FaaSSimulator:\n",
        "    \"\"\"\n",
        "    Simulates a FaaS environment.\n",
        "\n",
        "    Demonstrates:\n",
        "    - Function isolation\n",
        "    - Event-based invocation\n",
        "    - Resource management (simulated)\n",
        "    - Function routing\n",
        "\n",
        "    In real FaaS platforms (like AWS Lambda):\n",
        "    - Functions run in isolated containers\n",
        "    - Resources are automatically managed\n",
        "    - Scaling happens automatically\n",
        "    - Includes monitoring and logging\n",
        "    \"\"\"\n",
        "    def __init__(self):\n",
        "        self.indexer = IndexerFunction()\n",
        "        self.searcher = SearcherFunction(self.indexer)\n",
        "        self.invocations = 0\n",
        "\n",
        "    def invoke(self, function_name, event):\n",
        "        \"\"\"\n",
        "        Simulates FaaS function invocation\n",
        "\n",
        "        Args:\n",
        "            function_name (str): Name of function to invoke\n",
        "            event (dict): Event data for the function\n",
        "\n",
        "        Real FaaS differences:\n",
        "        - Would spawn new container/instance\n",
        "        - Would handle concurrent requests\n",
        "        - Would implement timeout limits\n",
        "        - Would include error handling\n",
        "        \"\"\"\n",
        "        self.invocations += 1\n",
        "\n",
        "        if function_name == 'indexer':\n",
        "            return self.indexer.handle(event)\n",
        "        elif function_name == 'searcher':\n",
        "            return self.searcher.handle(event)\n",
        "        else:\n",
        "            raise ValueError(f\"Unknown function: {function_name}\")\n",
        "\n",
        "def demonstrate_faas():\n",
        "    \"\"\"\n",
        "    Demonstrates key FaaS concepts through example usage\n",
        "\n",
        "    Shows:\n",
        "    1. Event-driven invocation\n",
        "    2. Function independence\n",
        "    3. Scalability potential\n",
        "    \"\"\"\n",
        "    # Test data setup\n",
        "    test_documents = [\n",
        "        {\n",
        "            'document_id': 'doc1',\n",
        "            'content': 'Python is a popular programming language for cloud computing'\n",
        "        },\n",
        "        {\n",
        "            'document_id': 'doc2',\n",
        "            'content': 'Cloud computing enables scalable microservices architecture'\n",
        "        }\n",
        "    ]\n",
        "\n",
        "    # Initialize FaaS environment\n",
        "    faas = FaaSSimulator()\n",
        "\n",
        "    # Demonstrate event-driven invocation\n",
        "    print(\"1. Event-Driven Invocation:\")\n",
        "    for doc in test_documents:\n",
        "        result = faas.invoke('indexer', doc)\n",
        "        print(f\"  Indexed document {doc['document_id']}: {result}\")\n",
        "\n",
        "    # Demonstrate independent function calls\n",
        "    print(\"\\n2. Independent Function Calls:\")\n",
        "    search_queries = [\n",
        "        {'query': 'cloud computing'},\n",
        "        {'query': 'python programming'}\n",
        "    ]\n",
        "\n",
        "    for query in search_queries:\n",
        "        result = faas.invoke('searcher', query)\n",
        "        print(f\"  Search results for '{query['query']}': {result}\")\n",
        "\n",
        "    # Demonstrate scalability concept\n",
        "    print(f\"\\n3. Scalability Demonstration:\")\n",
        "    print(f\"  Total function invocations: {faas.invocations}\")\n",
        "    print(\"  In real FaaS: These would execute in parallel with automatic scaling\")\n",
        "\n",
        "if __name__ == \"__main__\":\n",
        "    demonstrate_faas()\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "lAtm0xHKS3XO",
        "outputId": "36852487-60fa-403d-bbfa-6ef32c020e4f"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "1. Event-Driven Invocation:\n",
            "  Indexed document doc1: {'status': 'success', 'indexed_words': 9}\n",
            "  Indexed document doc2: {'status': 'success', 'indexed_words': 6}\n",
            "\n",
            "2. Independent Function Calls:\n",
            "  Search results for 'cloud computing': {'status': 'success', 'results': ['doc1', 'doc2']}\n",
            "  Search results for 'python programming': {'status': 'success', 'results': ['doc1']}\n",
            "\n",
            "3. Scalability Demonstration:\n",
            "  Total function invocations: 4\n",
            "  In real FaaS: These would execute in parallel with automatic scaling\n"
          ]
        }
      ]
    }
  ]
}

NameError: name 'null' is not defined